In [1]:
import pandas as pd

from dash import Dash, dcc, html, Input, Output         # pip install dash
import dash_bootstrap_components as dbc                 # pip install dash_bootstrap_components

import plotly.express as px
import plotly.graph_objects as go
import plotly.tools as tls
from plotly.subplots import make_subplots

import scipy.stats as stats

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import utils

In [2]:
crop_df = pd.read_csv('dashboard_data/fao_processed_unstacked.csv')
crop_df.head()


,Unnamed: 0,year,area,item,Area harvested,Production,Yield,Country,Alpha-3 code,Code
0,2,1961,Austria,Asparagus,35.0,150.0,42857.0,Austria,AUT,AUT
1,3,1961,Austria,Barley,187648.0,511571.0,27262.0,Austria,AUT,AUT
2,4,1961,Austria,"Beans, dry",682.0,1160.0,17009.0,Austria,AUT,AUT
3,6,1961,Austria,"Broad beans and horse beans, dry",659.0,1635.0,24810.0,Austria,AUT,AUT
4,8,1961,Austria,Buckwheat,224.0,256.0,11429.0,Austria,AUT,AUT


In [3]:
fao_raw_percentiles_df = pd.read_csv('dashboard_data/fao_raw_percentiles.csv')
fao_raw_percentiles_df.head()

,Unnamed: 0,domain,area,element,item,year,unit,value,flag_description,index,all_time_mean,percentile
0,0,Crops and livestock products,Portugal,Area harvested,"Almonds, in shell",1961,ha,37000,Estimated value,17,38632.74,50-75%
1,1,Crops and livestock products,Portugal,Area harvested,"Almonds, in shell",1962,ha,38000,Estimated value,17,38632.74,50-75%
2,2,Crops and livestock products,Portugal,Area harvested,"Almonds, in shell",1963,ha,36000,Estimated value,17,38632.74,50-75%
3,3,Crops and livestock products,Portugal,Area harvested,"Almonds, in shell",1964,ha,36000,Estimated value,17,38632.74,50-75%
4,4,Crops and livestock products,Portugal,Area harvested,"Almonds, in shell",1965,ha,38000,Estimated value,17,38632.74,50-75%


In [4]:
def filter_df(df, item, metric, area, year=0):
    return df[(df.value > 0.0) & (df.item == item) & (df.element == metric) & (df.area == area) & (df.year >= year)]

In [5]:
MODELS_LOADED_IN = utils.load_in_prophet_models('prophet_models')

### IMPORTANT: Please open in your browser at localhost:{PORT} to view the dashboard as when it would be deployed 

for example localhost:8078


In [61]:
# https://github.com/Coding-with-Adam/Dash-by-Plotly/blob/master/Bootstrap/bootstrap_intro.py

In [6]:
    
    
def create_map(df, crop, metric):
    data = df[df.item == crop]
   
    fig = px.choropleth(data,
        locations='Code',
        color=metric,
        animation_frame="year",
        hover_name='area'
    )
    
    fig.update_layout(
        title=dict(
            text='Crops',
            x=.5,
            font_size=18,
            ),
    
        geo=dict(bgcolor='#8ad6ff', lakecolor='#8ad6ff', projection_type='miller', scope='europe'),
        
        width = 500,
        height = 500,
        coloraxis=dict(colorscale="Reds", cmin=data[metric].min(), cmax=data[metric].max())
    )

    fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 60

    return fig


def predict_and_graph(df, item, metric, area, predict):
    data = df[(df.area == area) & (df.item == item)].reset_index().sort_values(by='year')
    
    n = f'{metric}_{area}_{item}'
    model = MODELS_LOADED_IN[n]

    if not model:
        print('could not load in model')
    
    predict = list(range(predict[0], predict[1]+1))
    predict = data.year.unique().tolist() + predict

    forecast = model.predict(pd.DataFrame(predict, columns=['ds']))    

    trace = go.Scatter(
        name = 'Actual',
        mode = 'markers',
        x = list(forecast['ds']),
        y = list(data[metric]),
        marker=dict(
            color='#FFBAD2',
            line=dict(width=1)
        )
   )
    
    trace1 = go.Scatter(
        name = 'Predicted Trend',
        mode = 'lines',
        x = list(forecast['ds']),
        y = list(forecast['yhat']),
        marker=dict(
            color='red',
            line=dict(width=3)
        )
    )

    upper_band = go.Scatter(
        name = 'upper band',
        mode = 'lines',
        x = list(forecast['ds']),
        y = list(forecast['yhat_upper']),
        line= dict(color='#57b88f'),
            fill = 'tonexty'
    )

    lower_band = go.Scatter(
        name= 'lower band',
        mode = 'lines',
        x = list(forecast['ds']),
        y = list(forecast['yhat_lower']),
        line= dict(color='#1705ff')
    )

    data = [trace1, lower_band, upper_band, trace]

    layout = dict(title=f'{item} in {area} {metric} estimates Using Meta Prophet univariate model',
             xaxis=dict(title = 'Dates', ticklen=2, zeroline=True))

    figure=dict(data=data,layout=layout)

    return figure

In [8]:
PORT = 8078

# Some bootstrap CSS 
app = Dash(__name__, external_stylesheets=[dbc.themes.ZEPHYR])

app.layout = dbc.Container([
    # Geoplot dropdows & graph
    dbc.Card([  
        dbc.Row([
            dbc.Col([
                html.H3(
                    children="EU-wide performance for selected crop & metric",
                    className="main_title",
                ),
                dbc.Card([
                    dcc.Dropdown(
                        id="crop",
                        options=crop_df.item.unique(),
                        value="Barley",
                    ),
                    dcc.Dropdown(
                        id="metric",
                        options=['Area Harvested', 'Production', 'Yield'],
                        value="Production",
                    ),
                ]),
                dcc.Graph(id="crop-eu-map"),
            ]),
            # Stats dropdowns & Graphs
            dbc.Col([   
                html.H3(
                    children="Statistical testing for differences between crop results in EU countries",
                    className="main_title",
                ),
                dbc.Row([ 
                    dbc.Col(
                    dcc.Dropdown(
                        id="stats-area1",
                        options=crop_df.area.unique(),
                        value="Ireland",
                    )),
                    dbc.Col(
                    dcc.Dropdown(
                        id="stats-area2",
                        options=crop_df.area.unique(),
                        value="Sweden",
                    )),
                    dbc.Col(
                    dcc.Dropdown(
                        id="stats-crop",
                        options=crop_df.item.unique(),
                        value="Barley",
                    )),
                    dbc.Col(
                    dcc.Dropdown(
                        id="stats-metric",
                        options=['Area Harvested', 'Production', 'Yield'],
                        value="Production",
                    )),
                    dbc.Col(
                    dcc.Dropdown(
                        id="stats-year",
                        options=crop_df.year.unique(),
                        value=1961,
                    )),
                ]),
                html.Div(id="stats-result"),
            ]),
        ]),
    ]),
    # Per country percentiles
    dbc.Card([ 
        dbc.Row([
            html.H3(
                children="All Crops Production for a specifid country",
                className="main_title",
            ),
            dbc.Row([
                dbc.Col(
                dcc.Dropdown(
                    id="area",
                    options=fao_raw_percentiles_df.area.unique(),
                    value="Ireland",
                )),
                dbc.Col(
                dcc.RadioItems(
                    utils.PERCENTILE_LABELS,
                    utils.PERCENTILE_LABELS[-1],
                    id="percentile",
                    inline=True,
                )),
            ]),
            dcc.Graph(id="line-graph"),
            dcc.Graph(id="bar-graph"),
        ]),
    ]),
    # Predictions
    dbc.Card([   
        html.H3(
            children="Future Esitmates",
            className="main_title",
        ),
        dbc.Row([ 
            dbc.Col(
            dcc.Dropdown(
                id="item",
                options=utils.TOP_IRISH_CROPS_PRODUCTION,
                value="Barley",
            )),
            dbc.Col(
            dcc.Dropdown(
                id="metric",
                options=['Yield', 'Production'],
                value="Production",
            )),
            dbc.Col(
            dcc.Dropdown(
                id="area",
                options=crop_df.area.unique(),
                value="Ireland",
            )),
        ]),
        dbc.Row(
            dcc.RangeSlider(2022, 2050, 1, 
                value=[2022, 2027], 
                marks={i: '{}'.format(i) for i in range(2022, 2050)},
                id='predict')),
        dcc.Graph(id="prediction-graph"),
    ]),
], style={'background': 'white', 'height': '100vh'})


@app.callback(
    Output("crop-eu-map", "figure"), 
    Input("crop", "value"),
    Input("metric", "value"))
def update_map(crop, metric):
    return create_map(crop_df, crop, metric)

@app.callback(
    Output("stats-result", "children"), 
    Input("stats-area1", "value"),
    Input("stats-area2", "value"),
    Input("stats-crop", "value"),
    Input("stats-metric", "value"),
    Input("stats-year", "value"))
def update_line_chart(area1, area2, crop, metric, year):
    X1 = filter_df(fao_raw_percentiles_df, crop, metric, area1, year)
    X2 = filter_df(fao_raw_percentiles_df, crop, metric, area2, year)

    t_test_barley = stats.ttest_ind(X1.value, X2.value, equal_var = False)
    return str(t_test_barley)


@app.callback([
    Output("line-graph", "figure"), 
    Output("bar-graph", "figure")], 
    Input("percentile", "value"),
    Input("area", "value"))
def update_line_chart(percentile, area):
    data=fao_raw_percentiles_df[(fao_raw_percentiles_df.value > 0.0) & (fao_raw_percentiles_df.unit == 't') & (fao_raw_percentiles_df.area == area) & (fao_raw_percentiles_df.percentile == percentile)]
    return (px.line(data, x="year", y="value", color='item', title=f"{percentile} Crops Production in {area}"), 
            px.bar(data, x="year", y="value", color="item", title=f"{percentile} Crops Production in {area}"))


@app.callback(
    Output("prediction-graph", "figure"),
    Input("item", "value"),
    Input("metric", "value"),
    Input("area", "value"),
    Input("predict", "value"),
    )
def update_prediction_chart(item, metric, area, predict):
    return predict_and_graph(crop_df, item, metric, area, predict)


app.run_server(debug=True, port=PORT)

---------------------------------------------------------------------------
DuplicateIdError                          Traceback (most recent call last)
DuplicateIdError: Duplicate component id found in the initial layout: `metric`

